scaricare hdf view richiede acco gratuito per i download

+ link manuale prisma obsoleti? https://wiki.earthdata.nasa.gov/display/DAS/Toolkit+Downloads  

+ **prisma usa librerie hdf-eos5 v1.13, hdf-eos5 è ora a versione 1.16**  

+ esempio di mmap su hdf5(non hdf-eos5) con julia https://github.com/JuliaIO/HDF5.jl/blob/master/test/mmap.jl  

+ immagine docker per hdf eos lib https://hub.docker.com/r/hdfeos/hdfeos  

+ sdp toolkit: https://wiki.earthdata.nasa.gov/display/DAS/SDP+Toolkit+and+HDF-EOS+EOSDIS++Core+System+Project  

+ tutorial (in c) https://portal.hdfgroup.org/display/HDF5/Learning+the+Basics

+ hdfview: tool da https://support.hdfgroup.org/ftp/HDF5/releases/HDF-JAVA/hdf-java-2.10.1/bin/ (plugin non sembra funzionare con versione più recente)
  plugin per hdf-eos da https://wiki.earthdata.nasa.gov/display/DAS/HDFView+HDF-EOS+Plug-in    
  guida installazione: https://newsroom.gsfc.nasa.gov/sdptoolkit/HDFView/README_HDFView_plugin_v2.10.html    
  
+ documentazione pkg hdf5 https://juliaio.github.io/HDF5.jl/stable/  

+ esempi manipulaz hdf5 in julia https://www.juliabloggers.com/hdf5-in-julia/  

+ pkg R per importare dati prisma l1/l2 https://lbusett.github.io/prismaread/  
+ https://portal.hdfgroup.org/display/support/HDF5%201.12.0

In [120]:
# docu package hdf5 per julia https://juliaio.github.io/HDF5.jl/stable/
# using Pkg
# pkg.add("HDF5")
using HDF5

#errori posson lasciare aperti file-> file inseriti in struct dinamica per poi esser facilmente chiusi tutti
openfiles = Dict()
counter = 0
# funzione che chiude tutti i file aperti
function closeall(dict)
    n = 0
    for (key, value) in dict
        close(pop!(dict, key))
        n=n+1;
    end
    counter = 0
    println("chiuso $n file")
end

closeall (generic function with 1 method)

apertura file con h5open(filename::string, mode::string) -> file   
mode:  

| mode | Meaning |
|---|---|
|"r"|read-only|
|"r+"|read-write, preserving any existing contents|
|"cw"|read-write, create file if not existing, preserve existing contents|
|"w"|read-write, destroying any existing contents (if any)|

In [121]:
testfilename = "test.h5"
fid = h5open(testfilename, "w") # crea file hdf5 vuoto (o lo apre e basta se già esiste)

close(fid)


In [122]:
close(fid)
rm(testfilename) #deleta file di prova

apre file
1. trova attributi file
2. per ogni attributo
    1. stampa nome, tipo, contenuto
2. trova sotto oggetti
3. per ogni sotto oggetto
    1. stampa nome, tipo
    2. trova attributi
    3. per ogni attrib
        1. stampa nome, tipo, contenuto
    5. se l'oggetto è un dataset, ne stampa le dimensioni ed il tipo di dati
    4. trova sotto oggetti (sse oggetto è gruppo, dataset non han sotto oggetti)    
    5. per ogni sotto oggetto
        1. vedi 4.

In [125]:
function viewfile(x)
    println("call viewfile")
   #=for i in x 
        if typeof(i)==HDF5Group
            viewgroup(i)
        else
            println(typeof(i))
        end
    end=#
    viewgroup(x)
    #dump(x)
end
    
function viewgroup(g)
    println("call viewgroup on $g")
    #nome
    #println(g)
    
    # attributi
    attributi = attrs(g)
    if isempty(names(attributi))
        println("niente attributi")
    else        
        viewattributes(attributi)
    end
    #attrs(obj) ritorna attributi oggetto
    nomi = names(g)
    # names(x::Union{HDF5Group,HDF5File}) ritorna array di nomi dei membri di obj
    println("nomi: $nomi")
    #view su oggetti contenuti
    for nome in nomi
        i = g[nome]
        ti = typeof(i)
        println("trovato sotto oggetto di nome: $nome e tipo $ti")
        if ti==HDF5Group            
            viewgroup(i)
        elseif ti==HDF5Dataset
            viewdata(i)
        else
            println(ti)
        end
    end
end

function viewdata(d)
    println("call viewdata on $d")
    
    #attributi
    attributi = attrs(d)
    if isempty(names(attributi))
        println("niente attributi")
    else        
        viewattributes(attributi)
    end
    
    # nome
    #println(d)
    #1 dataset mappabile?
    if ismmappable(d)
        println("dataset mmappabile")
    else
        println("dataset non mmappabile")
    end
    #mappatura dataset con readmmap() aumenta rpestazioni per letture numerose di un ds
    
    # datatype
    println("tipo dati: $(typeof(read(d)))")
    
    # dataspace (dimensioni )
    println("dimensioni: $(size(d))")
end 

function viewattributes(atts::HDF5.HDF5Attributes)
    println("call viewattributes on $atts")
    nomi = names(atts)
    for nome in nomi
        content = read(atts, nome)
        println("attributo di nome: $nome ; tipo: $(typeof(content))")
        println("contenuto: $content")
    end
end
            

viewattributes (generic function with 2 methods)

In [126]:
#apro in lettura un file sample preso da hdfviewer, faccio qualcosa, chiudo file
#inputfile = "OMI-Aura_.he5"
inputfile = "hdf5_test.h5"
println("ishdf5 ? $(ishdf5(inputfile))") #ishdf5 prende stringa (nome file) ritorna se file è hdf5
file2 = h5open(inputfile, "r")
openfiles["file $counter"]=file2
counter = counter+1
#foo(file2)
viewfile(file2)
closeall(openfiles)

ishdf5 ? true
call viewfile
call viewgroup on HDF5 data file: hdf5_test.h5
call viewattributes on HDF5.HDF5Attributes(HDF5 data file: hdf5_test.h5)
attributo di nome: test ; tipo: Array{String,1}
contenuto: ["this is a test     "]
nomi: ["A note", "Notes", "arrays", "datatypes", "images"]
trovato sotto oggetto di nome: A note e tipo HDF5Dataset
call viewdata on HDF5 dataset: /A note (file: hdf5_test.h5 xfer_mode: 0)
niente attributi
dataset non mmappabile
tipo dati: Array{String,1}
dimensioni: (2,)
trovato sotto oggetto di nome: Notes e tipo HDF5Dataset
call viewdata on HDF5 dataset: /Notes (file: hdf5_test.h5 xfer_mode: 0)
niente attributi
dataset non mmappabile
tipo dati: Array{String,2}
dimensioni: (5, 10)
trovato sotto oggetto di nome: arrays e tipo HDF5Group
call viewgroup on HDF5 group: /arrays (file: hdf5_test.h5)
niente attributi
nomi: ["1D String", "2D float array", "2D int array", "2D int8x9", "3D int array", "4D int", "ArrayOfStructures", "Vdata table: PerBlockMetadataCommon

In [119]:
openfiles

Dict{Any,Any}()

search: datatype DataType HDF5Datatype



No documentation found.

`HDF5.datatype` is a `Function`.

```
# 12 methods for generic function "datatype":
[1] datatype(str::String) in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1187
[2] datatype(dset::HDF5Attribute) in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1171
[3] datatype(dset::HDF5Dataset) in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1169
[4] datatype(x::Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj}) in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1174
[5] datatype(::Type{T}) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1175
[6] datatype(str::Array{S,N} where N) where S<:String in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1193
[7] datatype(A::AbstractArray{T,N} where N) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1176
[8] datatype(::Type{Complex{T}}) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1177
[9] datatype(x::Complex{T}) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1184
[10] datatype(A::AbstractArray{Complex{T},N} where N) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1185
[11] datatype(A::HDF5Vlen{T}) where T<:Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.HDF5ReferenceObj} in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1199
[12] datatype(str::HDF5Vlen{C}) where C<:HDF5.CharType in HDF5 at C:\Users\i am\.julia\packages\HDF5\T1b9x\src\HDF5.jl:1200
```
